# Article Categorizer

## Objective
This is an application tailored for retail companies that aim to seamlessly match articles on their
platform to corresponding products (for example, an article detailing the tap water quality issue
in the UK could be matched to products like water clean or filter). By leveraging the capabilities
of LLMs integrated within EvaDB, the application aims to provide an end-to-end experience to
generate accurate matches between the input articles and product categories, without requiring
implementing additional NLP or ML models.

## Methodology
- Article Summarization: Given an article of any length, utilize the LLM text summarization
integrated into EvaDB to generate concise summaries. This step ensures that the core essence of
the article is captured without being overwhelmed by the volume of content.
- Search Index: Construct a search index on both the article summarizations and the product
categories. This index will serve as the foundation for our matching algorithm.
- Initial Matching: For each article, identify the top matches (e.g., top 10) from the product
categories based on the search index. This step provides a preliminary list of potential
product-category matches.
- Refinement with ChatGPT
- : To ensure relevance and accuracy, employ LLM like ChatGPT
integrated within EvaDB to refine the matches. The goal is to select the most fitting category for
each article. If no suitable category is found, the application will return no suggestions rather
than providing irrelevant matches.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/JinchuLi2002/article-categorizer/report.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/JinchuLi2002/article-categorizer"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>

</table><br><br>

## Prerequisites: input your OPENAI API key

In [ ]:
import openai
import os

OPENAI_API_KEY = "sk-..." # change here

openai.api_key = OPENAI_API_KEY
os.environ['OPENAI_KEY'] = OPENAI_API_KEY

## Getting Started



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Replace 'path_to_your_folder' with the actual path of your folder inside Google Drive
path = "/content/drive/My Drive/Colab Notebooks/article-tagger"

os.chdir(path)

# Print current working directory to confirm
print(os.getcwd())

/content/drive/My Drive/Colab Notebooks/article-tagger


## Import Category Data
The data used in this application is the Amazon Categories List, retrieved from (https://www.asinspotlight.com/amz-categories-list-csv). The categories comes with a hierarchical structure, for simplicity, we will only use the 3rd level categories, since they provide sufficient granularity for matching yet not too specific. For this demonstration, we will only be using the first 10 categories in the list. But it can be set to any number of categories.



In [ ]:
import csv

filename = "AmazonCategories.csv"

third_level_categories = set()

with open(filename, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        if len(row) > 3 and row[2] and not row[3]:
            third_level_categories.add(row[2])

# Print the unique third-level categories
for category in list(third_level_categories)[:10]:
    print(category)


Religious Arts & Photography
Air Mattresses & Accessories
Special Interest
Christmas Collectible Buildings
Men's Shops
Xbox Series X & S Games
Men's Clothing
Kids' Play Trains & Trams
Automotive Replacement Caps
Office Presentation Products


## Install EvaDB

In [ ]:
%pip install --quiet "evadb[document]"
%pip install psycopg2

import evadb
cursor = evadb.connect().cursor()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.1/530.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━

## Create & Populate Categories Table


In [ ]:
cursor.query("""
  DROP TABLE IF EXISTS categories
""").df()
cursor.query("""
  CREATE TABLE categories (id INTEGER, category TEXT(30))
""").df()

,0
0,Table Successfully dropped: categories


In [ ]:

import re
import nest_asyncio

# Function to keep only English characters and spaces
def filter_english_chars(text):
    return re.sub(r'[^A-Za-z ]', '', text)

nest_asyncio.apply()

categories_list = list(third_level_categories)
# Filter each category to retain only English characters and spaces
filtered_categories = sorted([filter_english_chars(category) for category in categories_list])

data_to_insert = [(idx, category) for idx, category in enumerate(filtered_categories)][:10]

for idx, category in data_to_insert:
    # Use executemany to insert all rows
    cursor.query(f"INSERT INTO categories (id, category) VALUES ({idx}, '{category}')").df()


In [ ]:
cursor.query("SELECT * FROM categories").df()

,categories._row_id,categories.id,categories.category
0,1,0,AC Adapters
1,2,1,Abrasive Accessories
2,3,2,Abrasive Brushes
3,4,3,Abrasive Finishing Products
4,5,4,Abrasive Mounted Points
5,6,5,Abrasive Wheels Discs
6,7,6,Abuse SelfHelp
7,8,7,Accent Furniture
8,9,8,Accounting
9,10,9,Acoustic AcousticElectric Basses


## Create Index for Categories
In order to perform efficient search, we will create an index on the categories table. The index will be built on the OpenAI embeddings of the category names. The custom function used for EvaDB to create embedding for a given string is defined in openai_embedding_extractor.py

In [ ]:
cursor.query("DROP FUNCTION IF EXISTS OpenAIEmbeddingExtractor;").df()
cursor.query(f"""
CREATE FUNCTION IF NOT EXISTS OpenAIEmbeddingExtractor
IMPL './openai_embedding_extractor.py';
""").df()

,0
0,Function OpenAIEmbeddingExtractor successfully...


In [ ]:
cursor.query("DROP INDEX IF EXISTS index_table").df()
cursor.query("""
    CREATE INDEX index_table
    ON categories (OpenAIEmbeddingExtractor(category))
    USING FAISS;
""").df()

10-18-2023 01:20:29 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_index:0113] Index index_table does not exist, therefore cannot be dropped.


category    AC Adapters
Name: 0, dtype: object
category    Abrasive Accessories
Name: 1, dtype: object
category    Abrasive Brushes
Name: 2, dtype: object
category    Abrasive Finishing Products
Name: 3, dtype: object
category    Abrasive Mounted Points
Name: 4, dtype: object
category    Abrasive Wheels  Discs
Name: 5, dtype: object
category    Abuse SelfHelp
Name: 6, dtype: object
category    Accent Furniture
Name: 7, dtype: object
category    Accounting
Name: 8, dtype: object
category    Acoustic  AcousticElectric Basses
Name: 9, dtype: object


,0
0,Index index_table successfully added to the da...


## Creaate & Populate Articles Table
Now we will read in the articles from articles/ directory and populate the articles table. In addition, since articles can be of arbitrary length that may exceed the embedding limit of OpenAI, we will also generate summaries for each article using EvaDB.

In [ ]:
cursor.query("""
  DROP TABLE IF EXISTS articles
""").df()
cursor.query("""
  CREATE TABLE articles (id INTEGER, article TEXT(30000))
""").df()

,0
0,Table Successfully dropped: articles


In [ ]:
import os
import numpy as np
import re

articles_directory = 'articles/'

texts = []
for filename in os.listdir(articles_directory):
    if filename.endswith('.txt'):
        with open(os.path.join(articles_directory, filename), 'r') as file:
            text = file.read().replace("\n", " ")
            text = re.sub(r'[^A-Za-z ]', '', text)
            texts.append(text)

In [ ]:
for i, t in enumerate(texts):
  cursor.query(f"INSERT INTO articles (id, article) VALUES ({i}, '{t}')").df()

See contents of articles table below:

In [ ]:
cursor.query("SELECT * FROM articles").df()

,articles._row_id,articles.id,articles.article
0,1,0,SKIP TO CONTENTSKIP TO SITE INDEX SECTION NAV...
1,2,1,Skip To Content Wirecutter Log in Subscribe De...


Now we will generate summaries for each article.

In [ ]:
cursor.query("""
CREATE FUNCTION IF NOT EXISTS TextSummarizer
TYPE HuggingFace
TASK 'summarization'
MODEL 'facebook/bart-large-cnn';
""").df()

,0
0,"Function TextSummarizer already exists, nothin..."


In [ ]:
cursor.query(f"""
DROP TABLE IF EXISTS temp_summaries;
""").df()

cursor.query(f"""
CREATE TABLE temp_summaries AS
SELECT id, TextSummarizer(article)
FROM articles;
""").df()

cursor.query("""
DROP TABLE IF EXISTS articles_with_summaries;
""").df()

cursor.query("""
CREATE TABLE articles_with_summaries AS
SELECT a.id, a.article, t.summary_text
FROM articles AS a
JOIN temp_summaries AS t ON a.id = t.id;
""").df()

,0
0,The table temp_summaries has been successfully...


The articles_with_summaries table now contains the summaries for each article:

In [ ]:
cursor.query("SELECT * FROM articles_with_summaries").df()

,articles_with_summaries._row_id,articles_with_summaries.id,articles_with_summaries.article,articles_with_summaries.summary_text
0,1,0,SKIP TO CONTENTSKIP TO SITE INDEX SECTION NAV...,Gum disease has been associated with a range o...
1,2,1,Skip To Content Wirecutter Log in Subscribe De...,After years of research and testing were ready...


## Get Most Relevant Categories for Each Article
Now we will be using the built-in similarity search functionality of EvaDB. For each article, we will retrieve the top 5 most relevant categories based on the similarity between the embeddings of article summary and the category name. 

In [ ]:
# Create the result table structure
cursor.query("""
DROP TABLE IF EXISTS article_similar_categories;
""").df()

cursor.query("""
CREATE TABLE article_similar_categories (
    article_id INTEGER,
    summary TEXT(3000),
    category_1 TEXT(100),
    category_2 TEXT(100),
    category_3 TEXT(100),
    category_4 TEXT(100),
    category_5 TEXT(100)
);
""").df()

all_articles = cursor.query("SELECT id, summary_text FROM articles_with_summaries;").df()

for index, row in all_articles.iterrows():
    article_id = row[0]
    summary_text = row[1]

    similar_categories = cursor.query(f"""
    SELECT category FROM categories
    ORDER BY Similarity(
        OpenAIEmbeddingExtractor('{summary_text}'),
        OpenAIEmbeddingExtractor(category)
    )
    LIMIT 5;
    """).df()

    # Extracting top 5 categories. If there are fewer than 5 results, the rest will be set as None.
    cat_1 = similar_categories.iloc[0][0] if len(similar_categories) > 0 else None
    cat_2 = similar_categories.iloc[1][0] if len(similar_categories) > 1 else None
    cat_3 = similar_categories.iloc[2][0] if len(similar_categories) > 2 else None
    cat_4 = similar_categories.iloc[3][0] if len(similar_categories) > 3 else None
    cat_5 = similar_categories.iloc[4][0] if len(similar_categories) > 4 else None

    # Insert the results into the new table
    cursor.query(f"""
    INSERT INTO article_similar_categories (article_id, summary, category_1, category_2, category_3, category_4, category_5)
    VALUES ({article_id}, '{summary_text}', '{cat_1}', '{cat_2}', '{cat_3}', '{cat_4}', '{cat_5}');
    """).df()


0    Gum disease has been associated with a range o...
Name: 0, dtype: object
0    After years of research and testing were ready...
Name: 0, dtype: object


Run the below cell to see the top 5 categories for each article, category_1 represents the most relevant category, category_2 represents the second most relevant category, and so on.

In [ ]:
cursor.query("SELECT * FROM article_similar_categories;").df()

,another_article_similar_categories._row_id,another_article_similar_categories.article_id,another_article_similar_categories.summary,another_article_similar_categories.category_1,another_article_similar_categories.category_2,another_article_similar_categories.category_3,another_article_similar_categories.category_4,another_article_similar_categories.category_5
0,1,0,Gum disease has been associated with a range o...,Accounting,Abuse SelfHelp,Abrasive Brushes,Abrasive Accessories,Abrasive Mounted Points
1,2,1,After years of research and testing were ready...,Accent Furniture,Acoustic AcousticElectric Basses,Accounting,Abuse SelfHelp,AC Adapters


## Refine Matches with ChatGPT
Sometimes the content of an article simply does not match any of the categories in the product database. In our example, sofa.txt is related to the "Accent Furniture" category, which should be paired together. ps5.txt, however, does not match any of the 10 categories we have in this demo, neither of the 5 results returned by similarity search would be useful. 

To address this issue, we will use the ChatGPT API integrated within EvaDB to refine the matches. The goal is to select the most fitting category for each article. If no suitable category is found, the application will return no suggestions rather than providing irrelevant matches.

In [ ]:
cursor.query("DROP TABLE IF EXISTS article_final_category;").df()
cursor.query("""
CREATE TABLE article_final_category (
    article_id INTEGER,
    selected_category TEXT(300)
);
""").df()

all_articles = cursor.query("SELECT * FROM article_similar_categories;").df()

for index, row in all_articles.iterrows():
    article_id = row[1]
    summary_text = row[2]
    categories = [row[3], row[4], row[5], row[6], row[7]]

    prompt = (f"Given the summary: '{summary_text}', "
          f"please choose the category that most closely aligns with the topic. DO NOT OUTPUT A SENTENCE, JUST THE CATEGORY"
          f"If none are even remotely related, reply 'none'. "
          f"The available categories are: {', '.join([cat for cat in categories if cat])}.")


    category_choice = cursor.query(f"""
    SELECT ChatGPT("{prompt}")
    """).df().iloc[0][0]

    # Insert the result into the new table
    cursor.query(f"""
    INSERT INTO article_final_category (article_id, selected_category)
    VALUES ({article_id}, '{category_choice}');
    """).df()


8
8


Run the below cell to see the final result, sofa.txt is matched to Accent Furniture, and ps5.txt now is not matched to any category.

In [ ]:
cursor.query("SELECT * FROM article_final_category;").df()

,article_final_category._row_id,article_final_category.article_id,article_final_category.selected_category
0,1,0,none
1,2,1,Accent Furniture
